In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [3]:
# Whole dataset
pickle_in = open("/content/drive/MyDrive/IDSIA Biomedical Texts/AllSourceEmotions_TwentyJune.pickle", 'rb')
df = pickle.load(pickle_in)
df.head(4)

,urls,text,source,label,WC,Analytic,Clout,Authentic,Tone,WPS,...,pride,realization,relief,remorse,sadness,surprise,neutral,all_emolabelprob,all_emo_labels,all_emo_label_rank
0,https://www.quora.com/What-are-panic-attacks-l...,I have been dealing with these for quite some ...,Quora,1,607,55.22,35.35,48.82,1.00,26.39,...,0.000041,0.000323,0.001267,0.000330,0.000575,0.000205,0.000843,"[(fear, 0.8543168), (nervousness, 0.068863705)...","[fear, nervousness, confusion, curiosity, cari...","{'fear': 1, 'nervousness': 2, 'confusion': 3, ..."
1,https://www.quora.com/What-are-panic-attacks-l...,"I was walking to school, but was running late ...",Quora,1,401,54.08,1.31,99.00,8.57,13.83,...,0.000617,0.022932,0.000231,0.003261,0.416074,0.001216,0.003240,"[(sadness, 0.41607377), (annoyance, 0.20149426...","[sadness, annoyance, amusement, disappointment...","{'sadness': 1, 'annoyance': 2, 'amusement': 3,..."
2,https://www.quora.com/What-are-panic-attacks-l...,"For me, panic attacks come two ways: randomly ...",Quora,1,446,25.83,93.36,75.79,1.00,12.74,...,0.000224,0.336612,0.002241,0.001373,0.007805,0.013074,0.041802,"[(confusion, 0.4124936), (realization, 0.33661...","[confusion, realization, fear, neutral, nervou...","{'confusion': 1, 'realization': 2, 'fear': 3, ..."
3,https://www.quora.com/What-are-panic-attacks-l...,"I’m driving home, riding a tingly high from sa...",Quora,1,525,30.63,2.97,96.06,6.86,15.44,...,0.000740,0.000619,0.000275,0.001561,0.057985,0.000332,0.000073,"[(fear, 0.8212809), (sadness, 0.057984624), (a...","[fear, sadness, anger, grief, nervousness, car...","{'fear': 1, 'sadness': 2, 'anger': 3, 'grief':..."


In [4]:
len(df)

7405

### Converting all texts to small case because lexicon words are in small case

In [ ]:
df['text'] = df['text'].apply(str.lower)

### Importing Lexicons file

In [ ]:
lexicons_df = pd.read_csv('/content/drive/MyDrive/IDSIA Biomedical Texts/NRC-Emotion-Intensity-Lexicon-v1.txt', header=None, delimiter="\t")
lexicons_df.columns = ['lex_word', 'lex_emotion', 'lex_intensity']
lexicons_df

,lex_word,lex_emotion,lex_intensity
0,outraged,anger,0.964
1,brutality,anger,0.959
2,hatred,anger,0.953
3,hateful,anger,0.940
4,terrorize,anger,0.939
...,...,...,...
9824,fugitive,trust,0.141
9825,mistakes,trust,0.133
9826,bait,trust,0.133
9827,divorce,trust,0.133


In [ ]:
lexicons_df['lex_emotion'].value_counts()

fear            1763
trust           1490
anger           1481
sadness         1294
joy             1264
disgust         1092
anticipation     862
surprise         583
Name: lex_emotion, dtype: int64

In [ ]:
list(lexicons_df['lex_emotion'].unique())

['anger',
 'anticipation',
 'disgust',
 'fear',
 'joy',
 'sadness',
 'surprise',
 'trust']

In [ ]:
lexicons_df[lexicons_df['lex_emotion'] == 'anger']

,lex_word,lex_emotion,lex_intensity
0,outraged,anger,0.964
1,brutality,anger,0.959
2,hatred,anger,0.953
3,hateful,anger,0.940
4,terrorize,anger,0.939
...,...,...,...
1476,celebrity,anger,0.026
1477,sisterhood,anger,0.015
1478,autocorrect,anger,0.015
1479,musical,anger,0.011


In [ ]:
print(list(lexicons_df[lexicons_df['lex_emotion']=='sadness']['lex_word']))

['mourning', 'heartbreaking', 'tragic', 'holocaust', 'suicidal', 'misery', 'massacre', 'euthanasia', 'depression', 'grieving', 'bereavement', 'fatal', 'bereaved', 'devastation', 'death', 'devastated', 'suicide', 'catastrophe', 'horrifying', 'died', 'tragedy', 'depressing', 'anguish', 'agony', 'deadly', 'weeping', 'stillbirth', 'deceased', 'murderer', 'cancer', 'dying', 'rape', 'devastating', 'morbidity', 'sadness', 'execution', 'traumatic', 'perish', 'grief', 'abandonment', 'atrocity', 'crucifixion', 'depressed', 'cadaver', 'treachery', 'betrayed', 'grieve', 'suffering', 'funeral', 'destroyed', 'homicide', 'hopelessness', 'slaughtering', 'persecution', 'murderous', 'sorrow', 'miserable', 'sad', 'horrific', 'unhappiness', 'bloodshed', 'crippled', 'carnage', 'pained', 'manslaughter', 'unbearable', 'stillborn', 'morbid', 'annihilation', 'helplessness', 'mourn', 'murder', 'slavery', 'abandoned', 'horrors', 'mutilation', 'annihilated', 'casualty', 'enslaved', 'sickness', 'torment', 'miscarr

### Trying

In [ ]:
import pandas as pd

# Sample data (replace with your actual DataFrames)
data = {
    'text': ["The quick brown fox", "jumps over the lazy dog", "i was so outraged"]
}
df = pd.DataFrame(data)

lexicons_data = {
    'lex_word': ['outraged', 'mistakes', 'quick', 'lazy'],
    'lex_emotion': ['anger', 'trust', 'joy', 'trust'],
    'lex_intensity': [0.964, 0.725, 0.812, 0.918]
}
lexicons_df = pd.DataFrame(lexicons_data)

# Step 1: Create a dictionary to store emotion and cumulative intensity
emotion_intensity_dict = {emotion: [] for emotion in lexicons_df['lex_emotion'].unique()}

# Step 2: Loop through lexicons_df and update the emotion_intensity_dict
for _, row in lexicons_df.iterrows():
    emotion_intensity_dict[row['lex_emotion']].append(row['lex_intensity'])

# Step 3 & 4: Process each text row in df and calculate the average intensity for each emotion
result_dict_list = []
for _, row in df.iterrows():
    text = row['text']
    temp_intensity_dict = {emotion: [] for emotion in lexicons_df['lex_emotion'].unique()}

    # Tokenize the text into words
    words = text.split()

    # Check if the word is present in lexicons_df and update the temp_intensity_dict
    for word in words:
        matching_rows = lexicons_df[lexicons_df['lex_word'] == word]
        for _, match_row in matching_rows.iterrows():
            temp_intensity_dict[match_row['lex_emotion']].append(match_row['lex_intensity'])

    # Calculate the average intensity for each emotion
    average_intensity_dict = {
        emotion: sum(intensities) / len(intensities) if len(intensities) > 0 else 0
        for emotion, intensities in temp_intensity_dict.items()
    }

    result_dict_list.append(average_intensity_dict)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_dict_list)

# Fill any missing emotions with 0 intensity
for emotion in emotion_intensity_dict.keys():
    if emotion not in result_df.columns:
        result_df[emotion] = 0.0

# Reorder columns based on the original order in lexicons_df
result_df = result_df[lexicons_df['lex_emotion'].unique()]
result_df

,anger,trust,joy
0,0.000,0.000,0.812
1,0.000,0.918,0.000
2,0.964,0.000,0.000


In [ ]:
import pandas as pd

# Sample data (replace with your actual DataFrames)
# data = {
#     'text': ["The quick brown fox", "jumps over the lazy dog", "i was so outraged"]
# }
# df = pd.DataFrame(data)

df = df.head(10)

# lexicons_data = {
#     'lex_word': ['outraged', 'mistakes', 'quick', 'lazy'],
#     'lex_emotion': ['anger', 'trust', 'joy', 'trust'],
#     'lex_intensity': [0.964, 0.725, 0.812, 0.918]
# }
# lexicons_df = pd.DataFrame(lexicons_data)

# Step 1: Create a dictionary to store emotion and cumulative intensity
emotion_intensity_dict = {emotion: [] for emotion in lexicons_df['lex_emotion'].unique()}  # a dictionary

# Step 2: Loop through lexicons_df and update the emotion_intensity_dict
for _, row in lexicons_df.iterrows():
    emotion_intensity_dict[row['lex_emotion']].append(row['lex_intensity'])

# Step 3 & 4: Process each text row in df and calculate the average intensity for each emotion
result_dict_list = []
for _, row in df.iterrows():
    text = row['text']
    temp_intensity_dict = {emotion: [] for emotion in lexicons_df['lex_emotion'].unique()}

    # Tokenize the text into words
    words = text.split()

    # Check if the word is present in lexicons_df and update the temp_intensity_dict
    for word in words:
        matching_rows = lexicons_df[lexicons_df['lex_word'] == word]
        for _, match_row in matching_rows.iterrows():
            temp_intensity_dict[match_row['lex_emotion']].append(match_row['lex_intensity'])

    # Calculate the average intensity for each emotion
    average_intensity_dict = {
        emotion: sum(intensities) / len(intensities) if len(intensities) > 0 else 0
        for emotion, intensities in temp_intensity_dict.items()
    }

    result_dict_list.append(average_intensity_dict)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_dict_list)

# Fill any missing emotions with 0 intensity
for emotion in emotion_intensity_dict.keys():
    if emotion not in result_df.columns:
        result_df[emotion] = 0.0

# Reorder columns based on the original order in lexicons_df
result_df = result_df[lexicons_df['lex_emotion'].unique()]
result_df

In [ ]:
lexicons_df

,lex_word,lex_emotion,lex_intensity
0,outraged,anger,0.964
1,mistakes,trust,0.725
2,quick,joy,0.812
3,lazy,trust,0.918


In [ ]:
emotion_intensity_dict = {emotion: [] for emotion in lexicons_df['lex_emotion'].unique()}
emotion_intensity_dict

{'anger': [],
 'anticipation': [],
 'disgust': [],
 'fear': [],
 'joy': [],
 'sadness': [],
 'surprise': [],
 'trust': []}

In [ ]:
for _, row in lexicons_df.iterrows():
    emotion_intensity_dict[row['lex_emotion']].append(row['lex_intensity'])

In [ ]:
len(emotion_intensity_dict)

8

## Main Code

In [ ]:
# Whole dataset
pickle_in = open("/content/drive/MyDrive/IDSIA Biomedical Texts/AllSourceEmotions_TwentyJune.pickle", 'rb')
df = pickle.load(pickle_in)
df.head(4)

,urls,text,source,label,WC,Analytic,Clout,Authentic,Tone,WPS,...,pride,realization,relief,remorse,sadness,surprise,neutral,all_emolabelprob,all_emo_labels,all_emo_label_rank
0,https://www.quora.com/What-are-panic-attacks-l...,I have been dealing with these for quite some ...,Quora,1,607,55.22,35.35,48.82,1.00,26.39,...,0.000041,0.000323,0.001267,0.000330,0.000575,0.000205,0.000843,"[(fear, 0.8543168), (nervousness, 0.068863705)...","[fear, nervousness, confusion, curiosity, cari...","{'fear': 1, 'nervousness': 2, 'confusion': 3, ..."
1,https://www.quora.com/What-are-panic-attacks-l...,"I was walking to school, but was running late ...",Quora,1,401,54.08,1.31,99.00,8.57,13.83,...,0.000617,0.022932,0.000231,0.003261,0.416074,0.001216,0.003240,"[(sadness, 0.41607377), (annoyance, 0.20149426...","[sadness, annoyance, amusement, disappointment...","{'sadness': 1, 'annoyance': 2, 'amusement': 3,..."
2,https://www.quora.com/What-are-panic-attacks-l...,"For me, panic attacks come two ways: randomly ...",Quora,1,446,25.83,93.36,75.79,1.00,12.74,...,0.000224,0.336612,0.002241,0.001373,0.007805,0.013074,0.041802,"[(confusion, 0.4124936), (realization, 0.33661...","[confusion, realization, fear, neutral, nervou...","{'confusion': 1, 'realization': 2, 'fear': 3, ..."
3,https://www.quora.com/What-are-panic-attacks-l...,"I’m driving home, riding a tingly high from sa...",Quora,1,525,30.63,2.97,96.06,6.86,15.44,...,0.000740,0.000619,0.000275,0.001561,0.057985,0.000332,0.000073,"[(fear, 0.8212809), (sadness, 0.057984624), (a...","[fear, sadness, anger, grief, nervousness, car...","{'fear': 1, 'sadness': 2, 'anger': 3, 'grief':..."


In [ ]:
df['text'] = df['text'].apply(str.lower)

In [ ]:
lexicons_df = pd.read_csv('/content/drive/MyDrive/IDSIA Biomedical Texts/NRC-Emotion-Intensity-Lexicon-v1.txt', header=None, delimiter="\t")
lexicons_df.columns = ['lex_word', 'lex_emotion', 'lex_intensity']
lexicons_df

,lex_word,lex_emotion,lex_intensity
0,outraged,anger,0.964
1,brutality,anger,0.959
2,hatred,anger,0.953
3,hateful,anger,0.940
4,terrorize,anger,0.939
...,...,...,...
9824,fugitive,trust,0.141
9825,mistakes,trust,0.133
9826,bait,trust,0.133
9827,divorce,trust,0.133


In [ ]:
# took 16 mins to run with gpu

emotion_intensity_dict = {emotion: [] for emotion in lexicons_df['lex_emotion'].unique()} # creates an empty dictionary, store the intensities associated with each unique emotion present in lexicons_df. Eg. emotion_intensity_dict = { 'anger': [], 'trust': [], 'joy': [],... }

for _, row in lexicons_df.iterrows():
    emotion_intensity_dict[row['lex_emotion']].append(row['lex_intensity'])  # This loop iterates through each row in lexicons_df and updates the emotion_intensity_dict with the corresponding intensities for each emotion. For example, after this step, the dictionary would look like: emotion_intensity_dict = { 'anger': [0.964], 'trust': [0.725, 0.918], 'joy': [0.812] } (from above example)

result_dict_list = []
for _, row in df.iterrows():
    text = row['text']  # for a particular row (text actually)
    temp_intensity_dict = {emotion: [] for emotion in lexicons_df['lex_emotion'].unique()}  # same thing as above, creating empty dict to store emotions and adding up associated intensites when a word is found in lexicon_df

    words = text.split()  # Tokenize the text into words

    # Check if the word is present in lexicons_df and update the temp_intensity_dict
    for word in words:
        matching_rows = lexicons_df[lexicons_df['lex_word'] == word]
        for _, match_row in matching_rows.iterrows():
            temp_intensity_dict[match_row['lex_emotion']].append(match_row['lex_intensity'])

    # Calculate the average intensity for each emotion
    average_intensity_dict = {
        emotion: sum(intensities) / len(intensities) if len(intensities) > 0 else 0
        for emotion, intensities in temp_intensity_dict.items()
    }

    result_dict_list.append(average_intensity_dict)

result_df = pd.DataFrame(result_dict_list)

# Fill any missing emotions with 0 intensity
for emotion in emotion_intensity_dict.keys():
    if emotion not in result_df.columns:
        result_df[emotion] = 0.0

# Reorder columns based on the original order in lexicons_df
result_df = result_df[lexicons_df['lex_emotion'].unique()]
result_df

,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,0.415048,0.553423,0.272333,0.568205,0.409500,0.467625,0.434500,0.522773
1,0.530400,0.519750,0.541250,0.432167,0.453429,0.315600,0.247333,0.508875
2,0.428600,0.533500,0.228167,0.526192,0.413444,0.468533,0.348500,0.504500
3,0.567200,0.533462,0.114667,0.501952,0.505000,0.522095,0.320500,0.593615
4,0.487000,0.508000,0.482250,0.624833,0.489167,0.505333,0.000000,0.527167
...,...,...,...,...,...,...,...,...
7400,0.396000,0.609000,0.484000,0.527500,0.434000,0.591000,0.793000,0.540800
7401,0.000000,0.000000,0.000000,0.156000,0.000000,0.000000,0.000000,0.000000
7402,0.000000,0.000000,0.000000,0.156000,0.000000,0.000000,0.000000,0.641000
7403,0.344000,0.528667,0.000000,0.414000,0.515500,0.500000,0.363500,0.613000


In [ ]:
len(result_df)

7405

In [ ]:
result_df.rename(columns={'anger': 'anger_intensity',
                          'anticipation': 'anticipation_intensity',
                          'disgust': 'disgust_intensity',
                          'fear': 'fear_intensity',
                          'joy': 'joy_intensity',
                          'sadness': 'sadness_intensity',
                          'surprise': 'surprise_intensity',
                          'trust': 'trust_intensity'}, inplace=True)
result_df

,anger_intensity,anticipation_intensity,disgust_intensity,fear_intensity,joy_intensity,sadness_intensity,surprise_intensity,trust_intensity
0,0.415048,0.553423,0.272333,0.568205,0.409500,0.467625,0.434500,0.522773
1,0.530400,0.519750,0.541250,0.432167,0.453429,0.315600,0.247333,0.508875
2,0.428600,0.533500,0.228167,0.526192,0.413444,0.468533,0.348500,0.504500
3,0.567200,0.533462,0.114667,0.501952,0.505000,0.522095,0.320500,0.593615
4,0.487000,0.508000,0.482250,0.624833,0.489167,0.505333,0.000000,0.527167
...,...,...,...,...,...,...,...,...
7400,0.396000,0.609000,0.484000,0.527500,0.434000,0.591000,0.793000,0.540800
7401,0.000000,0.000000,0.000000,0.156000,0.000000,0.000000,0.000000,0.000000
7402,0.000000,0.000000,0.000000,0.156000,0.000000,0.000000,0.000000,0.641000
7403,0.344000,0.528667,0.000000,0.414000,0.515500,0.500000,0.363500,0.613000


In [ ]:
final_df = pd.concat([df, result_df], axis=1)
final_df

,urls,text,source,label,WC,Analytic,Clout,Authentic,Tone,WPS,...,all_emo_labels,all_emo_label_rank,anger_intensity,anticipation_intensity,disgust_intensity,fear_intensity,joy_intensity,sadness_intensity,surprise_intensity,trust_intensity
0,https://www.quora.com/What-are-panic-attacks-l...,i have been dealing with these for quite some ...,Quora,1,607,55.22,35.35,48.82,1.00,26.39,...,"[fear, nervousness, confusion, curiosity, cari...","{'fear': 1, 'nervousness': 2, 'confusion': 3, ...",0.415048,0.553423,0.272333,0.568205,0.409500,0.467625,0.434500,0.522773
1,https://www.quora.com/What-are-panic-attacks-l...,"i was walking to school, but was running late ...",Quora,1,401,54.08,1.31,99.00,8.57,13.83,...,"[sadness, annoyance, amusement, disappointment...","{'sadness': 1, 'annoyance': 2, 'amusement': 3,...",0.530400,0.519750,0.541250,0.432167,0.453429,0.315600,0.247333,0.508875
2,https://www.quora.com/What-are-panic-attacks-l...,"for me, panic attacks come two ways: randomly ...",Quora,1,446,25.83,93.36,75.79,1.00,12.74,...,"[confusion, realization, fear, neutral, nervou...","{'confusion': 1, 'realization': 2, 'fear': 3, ...",0.428600,0.533500,0.228167,0.526192,0.413444,0.468533,0.348500,0.504500
3,https://www.quora.com/What-are-panic-attacks-l...,"i’m driving home, riding a tingly high from sa...",Quora,1,525,30.63,2.97,96.06,6.86,15.44,...,"[fear, sadness, anger, grief, nervousness, car...","{'fear': 1, 'sadness': 2, 'anger': 3, 'grief':...",0.567200,0.533462,0.114667,0.501952,0.505000,0.522095,0.320500,0.593615
4,https://www.quora.com/What-are-panic-attacks-l...,it feels like your body turns into a black hol...,Quora,1,323,21.57,1.00,99.00,1.12,13.46,...,"[fear, nervousness, sadness, confusion, disapp...","{'fear': 1, 'nervousness': 2, 'sadness': 3, 'c...",0.487000,0.508000,0.482250,0.624833,0.489167,0.505333,0.000000,0.527167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7400,NaN,"hi ttrung,\nyour story is very similar to my o...",Beyond Blue Forums,0,237,5.24,20.52,53.23,15.51,16.93,...,"[gratitude, caring, admiration, relief, approv...","{'gratitude': 1, 'caring': 2, 'admiration': 3,...",0.396000,0.609000,0.484000,0.527500,0.434000,0.591000,0.793000,0.540800
7401,NaN,hi startingnew \nthanks for replying i am tryi...,Beyond Blue Forums,0,20,39.70,1.00,28.56,99.00,20.00,...,"[gratitude, neutral, approval, admiration, cur...","{'gratitude': 1, 'neutral': 2, 'approval': 3, ...",0.000000,0.000000,0.000000,0.156000,0.000000,0.000000,0.000000,0.000000
7402,NaN,hi mspurple \nthanks for replying i am trying ...,Beyond Blue Forums,0,19,36.67,3.34,70.28,99.00,19.00,...,"[gratitude, curiosity, approval, neutral, cari...","{'gratitude': 1, 'curiosity': 2, 'approval': 3...",0.000000,0.000000,0.000000,0.156000,0.000000,0.000000,0.000000,0.641000
7403,NaN,hi t4 \nthats ok many many people are on medic...,Beyond Blue Forums,0,38,36.67,90.88,33.61,92.27,19.00,...,"[caring, nervousness, optimism, approval, fear...","{'caring': 1, 'nervousness': 2, 'optimism': 3,...",0.344000,0.528667,0.000000,0.414000,0.515500,0.500000,0.363500,0.613000


In [ ]:
final_df.to_csv('AllSource_Intensity_ThirdJuly.csv', index=False)